In [1]:
import sys
from PyQt5.QtWidgets import *
from PyQt5.QtCore import pyqtSignal
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import phoneBook #модуль главного окна PyQt
import add_new #модель виджета добавления нового контакта PyQt
import mainModel #модель классов и методов для телеф книги
from PyQt5.QtCore import pyqtSignal
from PyQt5 import QtCore

class phoneBookMain(QMainWindow, phoneBook.Ui_MainWindow):
    def __init__(self, parent = None):
        super().__init__()
        self.setupUi(self)

        self.phonelist = mainModel.PhoneBook()# создаю экземпляр класса для списка контактов
        self.control = mainModel.Controller(self.phonelist) #создаю объект класса контроллера

        self.pushButton.clicked.connect(self.newContact) #запуск виджета addNew для создания нового контакта
        self.pushButton_2.clicked.connect(self.delSearch) #Очистка поля поиска
        self.pushButton_3.setEnabled(False)
        self.pushButton_3.clicked.connect(self.edit) #Редактирование
        self.pushButton_4.clicked.connect(self.Search) #Очитска поля поиска
        self.pushButton_5.clicked.connect(self.readFile) #Читаем из файла
        self.pushButton_6.clicked.connect(self.saveFile) #Записываем в файл
        self.pushButton_7.setEnabled(False)
        #self.pushButton_7.clicked.connect(self.deleteContact)

        self.listWidget.itemActivated.connect(self.itemActivated_event) #активация элемента телеф книги кликом мыши 
        
    def toDisplay(self):
        self.listWidget.clear()
        self.listWidget_2.clear()
        self.listWidget_3.clear()
        self.listWidget_4.clear()
        self.control.sortByName()
        for item in self.phonelist.items:
            self.listWidget.addItem(item.name)
        
    def newContact(self):        
        self.addContact = addNew()
        self.addContact.sendData.connect(self.createContact)
        self.addContact.show()
        
    def createContact(self, name = None, phones = None, email = None, idContact = None):
        self.control.addSimplePerson(name, phones, email, idContact)
        self.toDisplay()
        self.statusTimer()
        self.statusBar.showMessage('Contact added')
            
    def itemActivated_event(self, item):#метод для вывода полной инфо о контакте в виджет listWidget_2        
        self.listWidget_2.clear()

        personName = item.text()
        #personId = item.text(idContact)
        #print(personId)
        
        searchRes = self.control.search(personName)#ищем по имени инфо о контакте

        self.listWidget_2.addItem(searchRes.name)#выводим полную инфу о контакте в виджет
        for i in searchRes.phones:
            self.listWidget_3.addItem(str(i))            
        self.listWidget_4.addItem(searchRes.email)
        
        #self.listWidget_2.addItem(searchRes.idContact) 
        
        self.forEdit = searchRes.name
        self.idContactEdit = searchRes.idContact
        self.pushButton_3.setEnabled(True)
        self.pushButton_7.setEnabled(True)
        
        #print(self.forEdit, self.idContactEdit)
    
    def readFile(self):
        file, _ = QFileDialog.getOpenFileName(self, "Upload Contacts")
        if len(file) > 0:
            reading = self.control.readFile(file)
            self.toDisplay()
            
    def saveFile(self):
        filename, _ = QFileDialog.getSaveFileName(self, "Backup Contacts", "*.csv")
        if len(filename) > 0:
            self.control.addToFile(filename)
        self.statusTimer()
        self.statusBar.showMessage('File saved')
            
    def Search(self):
        self.listWidget.clear()
        self.listWidget_2.clear()

        name = self.lineEdit.text()
        #personId = item.text(idContact)
        #print(personId)
        
        searchRes = self.control.search_with_answ(name)#ищем по имени инфо о контакте

        if searchRes != False:
            for item in searchRes:
                self.listWidget.addItem(item.name)
        else:
            self.statusTimer()
            self.statusBar.showMessage('Not Found')
    
    def statusTimer(self):
        self.timer = QtCore.QTimer()
        self.timer.start(3000)
        self.timer.timeout.connect(self.clearStatusBar)
            
    def clearStatusBar(self):
        self.statusBar.clearMessage()
        
    def delSearch(self):
        self.lineEdit.clear()
        self.pushButton_3.setEnabled(False)
        self.pushButton_7.setEnabled(False)
        #self.listWidget.clear()
        #self.listWidget_2.clear()
        #self.listWidget_3.clear()
        #self.listWidget_4.clear()
        self.toDisplay()
                
    def keyPressEvent(self, e):
        if e.key() == Qt.Key_Escape:
            self.close()
            
    def edit(self):
        searchEdit = self.control.searchForEdit(self.forEdit, self.idContactEdit)

        self.editContact = editContact(searchEdit.name, searchEdit.phones, searchEdit.email, searchEdit.idContact)
        self.editContact.sendEditedData.connect(self.editSingleContact)        
        self.editContact.show()
    
    def editSingleContact(self, name = None, phones = None, email = None, idContact = None):
        searchEdit = self.control.searchById(idContact)

        res = self.control.editContact(name, phones, email, idContact)

        self.toDisplay()        
        self.listWidget_2.addItem(res.name)
        for i in res.phones:
            self.listWidget_3.addItem(str(i))            
        self.listWidget_4.addItem(res.email)
        #self.control.addSimplePerson(name, phones, email, idContact)

                
class addNew(QWidget, add_new.Ui_Form):
    sendData = QtCore.pyqtSignal(str, list, str, str)
 
    def __init__(self):
        super().__init__()        
        self.setupUi(self)  
        
        #Действия по кнопкам
        self.buttonBox.rejected.connect(self.window().close) 
        self.buttonBox.accepted.connect(self.addPerson)
        
    def keyPressEvent(self, e):
        if e.key() == Qt.Key_Escape:
            self.close()
        
    def addPerson(self):        
        p0 = mainModel.Phone(self.lineEdit_2.text(),int(self.comboBox_2.currentIndex()))#телефоны
        p1 = mainModel.Phone(self.lineEdit_3.text(),int(self.comboBox_3.currentIndex()))#
        p2 = mainModel.Phone(self.lineEdit_4.text(),int(self.comboBox_4.currentIndex()))#
        p3 = mainModel.Phone(self.lineEdit_5.text(),int(self.comboBox_5.currentIndex()))#
        phones = []
        phone = [p0, p1, p2, p3]
        
        name = self.lineEdit.text()
        for i in range(len(phone)):
            if len(phone[i].number) != 0:
                phones.append(phone[i])
        email = self.lineEdit_6.text()
        
        idContact = str(id(name))
        
        self.sendData.emit(name, phones, email, idContact)
        self.closeForm()

    def closeForm(self):          
        self.lineEdit.clear()
        self.lineEdit_2.clear()
        self.lineEdit_3.clear()
        self.lineEdit_4.clear()
        self.lineEdit_5.clear()
        self.lineEdit_6.clear()

        self.close()
        self.destroy()
        
class editContact(QWidget, add_new.Ui_Form):
    sendEditedData = QtCore.pyqtSignal(str, list, str, str)
 
    def __init__(self, name, phones, email, idContact):
        super().__init__()        
        self.setupUi(self) 
        self.setWindowTitle("Edit Contact")
        self.name = name
        self.phones = phones
        self.email = email
        self.idContact = idContact
        
        if len(self.name) != 0:
            self.lineEdit.setText(self.name)
        if len(self.phones) == 1:
            self.lineEdit_2.setText(self.phones[0].number), self.comboBox_2.setCurrentIndex(self.phones[0].kind)#телефоны
        if len(self.phones) == 2:
            self.lineEdit_2.setText(self.phones[0].number), self.comboBox_2.setCurrentIndex(self.phones[0].kind)
            self.lineEdit_3.setText(self.phones[1].number), self.comboBox_3.setCurrentIndex(self.phones[1].kind)#
        if len(self.phones) == 3:
            self.lineEdit_2.setText(self.phones[0].number), self.comboBox_2.setCurrentIndex(self.phones[0].kind)
            self.lineEdit_3.setText(self.phones[1].number), self.comboBox_3.setCurrentIndex(self.phones[1].kind)
            self.lineEdit_4.setText(self.phones[2].number), self.comboBox_4.setCurrentIndex(self.phones[2].kind)#
        if len(self.phones) == 4:
            self.lineEdit_2.setText(self.phones[0].number), self.comboBox_2.setCurrentIndex(self.phones[0].kind)
            self.lineEdit_3.setText(self.phones[1].number), self.comboBox_3.setCurrentIndex(self.phones[1].kind)
            self.lineEdit_4.setText(self.phones[2].number), self.comboBox_4.setCurrentIndex(self.phones[2].kind)
            self.lineEdit_5.setText(self.phones[3].number), self.comboBox_5.setCurrentIndex(self.phones[3].kind)            
        if len(self.email) != 0:
            self.lineEdit_6.setText(self.email)
            
        #Действия по кнопкам
        self.buttonBox.rejected.connect(self.window().close) 
        self.buttonBox.accepted.connect(self.editData)
        
    def keyPressEvent(self, e):
        if e.key() == Qt.Key_Escape:
            self.close()
            
    def editData(self):        
        p0 = mainModel.Phone(self.lineEdit_2.text(),int(self.comboBox_2.currentIndex()))#телефоны
        p1 = mainModel.Phone(self.lineEdit_3.text(),int(self.comboBox_3.currentIndex()))#
        p2 = mainModel.Phone(self.lineEdit_4.text(),int(self.comboBox_4.currentIndex()))#
        p3 = mainModel.Phone(self.lineEdit_5.text(),int(self.comboBox_5.currentIndex()))#
        
        phones = []
        phone = [p0, p1, p2, p3]
        
        name = self.lineEdit.text()
        for i in range(len(phone)):
            if len(phone[i].number) != 0:
                phones.append(phone[i])
        email = self.lineEdit_6.text()
        
        #print(name)
        #print(phones)
        #print(email)
        #print(self.idContact)
        
        self.sendEditedData.emit(name, phones, email, self.idContact)
        self.closeForm()

    def closeForm(self):          
        self.lineEdit.clear()
        self.lineEdit_2.clear()
        self.lineEdit_3.clear()
        self.lineEdit_4.clear()
        self.lineEdit_5.clear()
        self.lineEdit_6.clear()

        self.close()
        self.destroy()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = phoneBookMain()
    ex.show()
    app.exec()